In [1]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Construct_the_data_add_missing_values').getOrCreate()
df = spark.read.csv('../Bike-Sharing-Dataset-hour.csv', header=True, inferSchema=True)

In [2]:
df.createOrReplaceTempView("dataset")
#query to retrieve the data for the specific year and month
apr2011_df = spark.sql("SELECT instant, dteday, yr, mnth, hr FROM dataset WHERE yr = 0 AND mnth = 4 ORDER BY instant ")

listHrs = [-1] * 720 # define the full range of the hours
listDate = [-1] * 720 # define the full range of the date based on the hours
listMissingHrs = [] # define the missing hours
listMissingHrsOfDate = [] # define the date for the missing hours
listMissingWeekday = [] # define the weekday for the missing hours
listMissingWorkingday = [] # define the workingday for the missing hours

x = 0
for row in apr2011_df.collect(): # for loop to get the resultset into the list
    listDate[x] = str(row[1])
    listHrs[x] = str(row[4])
    x += 1

In [3]:
from datetime import datetime
a = 0
strHr = ''
for i in range(1, 31): # there are total days in the specific year and month starting at 1st day
    for j in range(0, 24): # there are 24 hours a day starting at 0 hour
        if i == int(listDate[a][8:10]) and j == int(listHrs[a]): # pass if date and hour are in the listHrs and listDate
            a += 1
        else: # add values into listMissingHrs and listMissingHrsOfDate if hour and hour of date are missing
            strHr = str(i)
            if i < 10:
                strHr = '0' + strHr
            strDate = "2011-04-"+strHr
            listMissingHrsOfDate.append(strDate)
            listMissingHrs.append(j)
            
            strWeekday = datetime.strptime(strDate, '%Y-%m-%d') # convert string time to date format
            listMissingWeekday.append(strWeekday.strftime("%w")) # retrieve the weekday of the date
            if strWeekday.strftime("%w") == '0' or strWeekday.strftime("%w") == '6': # workingday is 0 if Sun or Sat
                listMissingWorkingday.append(0)
            else: # workingday is 1 if it's not Sun or Sat
                listMissingWorkingday.append(1)

In [4]:
import csv
with open('../Bike-Sharing-Dataset-hour.csv','a') as fd: # open the existing csv file to be edited
    for m in range(len(listMissingHrsOfDate)): # for loop the list and add the missing values
        fd.write("0,"+str(listMissingHrsOfDate[m])+",2,0,4,"+str(listMissingHrs[m])+","
                 "0,"+str(listMissingWeekday[m])+","+str(listMissingWorkingday[m])+","
                 "2,0.4969,0.4757,0.6272,0.1900,35,153,188\n")